In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [4]:
tokenizer.pad_token = tokenizer.eos_token

In [5]:
def chat(user_input):
    inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=50, 
        temperature=0.7,  
        top_p=0.9,        
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)
    return response

In [ ]:
user_input = "What is AI?"
response = chat(user_input)
print(f"Anda: {user_input}")
print(f"Assistant: {response}")